In [1]:
import numpy as np
import pandas as pd
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras import regularizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures

# 数据整理
data = pd.read_csv('/Users/xiaoquanliu/Desktop/Book_DataCode/第七章/DL_Data2.csv')
data1=data.dropna()

features = data1.iloc[:, :-1] 
target = data1.iloc[:, -1]
from sklearn.preprocessing import PolynomialFeatures


# 创建一个 PolynomialFeatures 对象，设置 degree=2 来生成两两交互的特征
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

# 使用 PolynomialFeatures 对象转换你的数据
features_poly = poly.fit_transform(features)

# 将交互特征转换为 DataFrame，并添加列名
features_poly_df = pd.DataFrame(features_poly, columns=poly.get_feature_names_out(features.columns))

# 打印结果
print(features_poly_df)

# features_poly_df 包含了原始特征和它们的两两交互项

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features_poly_df)
features_train, features_test, target_train, target_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

input_dim = features_train.shape[1]  



2024-09-15 21:45:23.792334: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


       Volatility     Beta1     Beta2      Cor1      Cor2  NonSysRisk1  \
0        0.583365  0.716008  0.726709  0.184198  0.188873     0.012349   
1        0.383995  0.913849  0.923003  0.409555  0.416662     0.025676   
2        0.423533  0.806881  0.831737  0.278495  0.292493     0.014550   
3        0.518040  0.943050  0.929599  0.254375  0.254799     0.018665   
4        0.550201  1.066480  1.068627  0.284123  0.290532     0.025388   
...           ...       ...       ...       ...       ...          ...   
78552    0.555577  0.944254  0.961975  0.247322  0.256579     0.019359   
78553    0.471594  1.882598  1.846909  0.568810  0.567869     0.074968   
78554    0.649889  1.180658  1.213428  0.317196  0.328823     0.042696   
78555    0.632866  1.286845  1.315512  0.349781  0.361143     0.049497   
78556    0.645599  1.472637  1.493594  0.335231  0.346022     0.047210   

       NonSysRisk2      Rsq1      Rsq2     ARsq1  ...  \
0         0.012984  0.033929  0.035673  0.030033  ... 

In [2]:
#设置隐藏因子的数量
encoding_dim = 120
hidden_dim=315
num_hidden_layer=20


# 添加L1正则化
encoder_layer = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(0.01 ))

# 添加L2正则化
encoder_layer = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l2(0.01))

decoding_dim=256
input_layer = Input(shape=(630,))
encoder_layer = Dense(encoding_dim, activation='relu', kernel_initializer='he_normal')(input_layer)
encoder = Model(inputs=input_layer, outputs=encoder_layer)

# 自编码器模型
autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(Dense(input_dim, activation='sigmoid'))  # 解码层

autoencoder.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error')



# 训练自编码器模型，使用训练集的特征进行训练和验证
autoencoder.fit(features_train, features_train, epochs=100,batch_size=512, shuffle=True, validation_data=(features_test, features_test))

# 使用编码器对特征进行降维
encoded_features_train = encoder.predict(features_train)
encoded_features_test = encoder.predict(features_test)

# 使用编码器对特征进行降维
encoded_features_train = encoder.predict(features_train)

# 转换为Pandas DataFrame
encoded_features_train_df = pd.DataFrame(encoded_features_train)


# 使用降维后的特征训练预测模型，这里我们使用一个简单的线性回归模型作为例子
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(encoded_features_train, target_train)

# 对测试集进行预测并评估模型性能
predictions = regressor.predict(encoded_features_test)

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(target_test, predictions)
print('Mean Squared Error on test set: ', mse)

from sklearn.linear_model import LinearRegression

# 使用降维后的特征训练线性回归模型
regressor = LinearRegression()
regressor.fit(encoded_features_train, target_train)

# 获取线性回归模型的参数
coefficients = regressor.coef_  # 回归系数，对应每个特征的权重
intercept = regressor.intercept_  # 截距项

# 打印每个特征的回归系数
for i, coef in enumerate(coefficients):
    print(f'Coefficient for encoded feature {i+1}: {coef}')

# 打印截距项
print(f'Intercept: {intercept}')

import statsmodels.api as sm

# 添加常数项，对应线性回归模型的截距
encoded_features_train_with_intercept = sm.add_constant(encoded_features_train)

# 使用statsmodels的OLS函数训练线性回归模型
model = sm.OLS(target_train, encoded_features_train_with_intercept)
results = model.fit()

# 打印每个特征的回归系数、标准误差和p值
print(results.summary())



Epoch 1/100


2024-09-15 21:45:31.590287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


123/123 [==============================] - 1s 5ms/step - loss: 0.1637 - val_loss: 0.1603
Epoch 2/100
123/123 [==============================] - 1s 4ms/step - loss: 0.1561 - val_loss: 0.1509
Epoch 3/100
123/123 [==============================] - 1s 4ms/step - loss: 0.1439 - val_loss: 0.1359
Epoch 4/100
123/123 [==============================] - 1s 4ms/step - loss: 0.1266 - val_loss: 0.1166
Epoch 5/100
123/123 [==============================] - 1s 4ms/step - loss: 0.1062 - val_loss: 0.0957
Epoch 6/100
123/123 [==============================] - 1s 4ms/step - loss: 0.0858 - val_loss: 0.0761
Epoch 7/100
123/123 [==============================] - 1s 4ms/step - loss: 0.0675 - val_loss: 0.0593
Epoch 8/100
123/123 [==============================] - 1s 4ms/step - loss: 0.0524 - val_loss: 0.0460
Epoch 9/100
123/123 [==============================] - 1s 4ms/step - loss: 0.0406 - val_loss: 0.0356
Epoch 10/100
123/123 [==============================] - 1s 4ms/step - loss: 0.0316 - val_loss: 0.0278
E

In [3]:
#设置隐藏因子的数量
encoding_dim = 240
hidden_dim=315
num_hidden_layer=60


# 添加L1正则化
encoder_layer = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(0.01 ))

# 添加L2正则化
encoder_layer = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l2(0.01))

decoding_dim=256
input_layer = Input(shape=(630,))
encoder_layer = Dense(encoding_dim, activation='relu', kernel_initializer='he_normal')(input_layer)
encoder = Model(inputs=input_layer, outputs=encoder_layer)

# 自编码器模型
autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(Dense(input_dim, activation='sigmoid'))  # 解码层

autoencoder.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error')


# 训练自编码器模型，使用训练集的特征进行训练和验证
autoencoder.fit(features_train, features_train, epochs=100,batch_size=512, shuffle=True, validation_data=(features_test, features_test))

# 使用编码器对特征进行降维
encoded_features_train = encoder.predict(features_train)
encoded_features_test = encoder.predict(features_test)

# 使用编码器对特征进行降维
encoded_features_train = encoder.predict(features_train)

# 转换为Pandas DataFrame
encoded_features_train_df = pd.DataFrame(encoded_features_train)


# 使用降维后的特征训练预测模型，这里我们使用一个简单的线性回归模型作为例子
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(encoded_features_train, target_train)

# 对测试集进行预测并评估模型性能
predictions = regressor.predict(encoded_features_test)

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(target_test, predictions)
print('Mean Squared Error on test set: ', mse)

from sklearn.linear_model import LinearRegression

# 使用降维后的特征训练线性回归模型
regressor = LinearRegression()
regressor.fit(encoded_features_train, target_train)

# 获取线性回归模型的参数
coefficients = regressor.coef_  # 回归系数，对应每个特征的权重
intercept = regressor.intercept_  # 截距项

# 打印每个特征的回归系数
for i, coef in enumerate(coefficients):
    print(f'Coefficient for encoded feature {i+1}: {coef}')

# 打印截距项
print(f'Intercept: {intercept}')

import statsmodels.api as sm

# 添加常数项，对应线性回归模型的截距
encoded_features_train_with_intercept = sm.add_constant(encoded_features_train)

# 使用statsmodels的OLS函数训练线性回归模型
model = sm.OLS(target_train, encoded_features_train_with_intercept)
results = model.fit()

# 打印每个特征的回归系数、标准误差和p值
print(results.summary())


Epoch 1/100
123/123 [==============================] - 1s 7ms/step - loss: 0.1604 - val_loss: 0.1541
Epoch 2/100
123/123 [==============================] - 1s 7ms/step - loss: 0.1452 - val_loss: 0.1344
Epoch 3/100
123/123 [==============================] - 1s 7ms/step - loss: 0.1211 - val_loss: 0.1065
Epoch 4/100
123/123 [==============================] - 1s 7ms/step - loss: 0.0916 - val_loss: 0.0767
Epoch 5/100
123/123 [==============================] - 1s 7ms/step - loss: 0.0640 - val_loss: 0.0523
Epoch 6/100
123/123 [==============================] - 1s 7ms/step - loss: 0.0433 - val_loss: 0.0353
Epoch 7/100
123/123 [==============================] - 1s 7ms/step - loss: 0.0294 - val_loss: 0.0242
Epoch 8/100
123/123 [==============================] - 1s 7ms/step - loss: 0.0205 - val_loss: 0.0173
Epoch 9/100
123/123 [==============================] - 1s 7ms/step - loss: 0.0150 - val_loss: 0.0131
Epoch 10/100
123/123 [==============================] - 1s 7ms/step - loss: 0.0116 - val_lo

In [4]:
#设置隐藏因子的数量
encoding_dim = 30
hidden_dim=315
num_hidden_layer=20


# 添加L1正则化
encoder_layer = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(0.01 ))

# 添加L2正则化
encoder_layer = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l2(0.01))

decoding_dim=256
input_layer = Input(shape=(630,))
encoder_layer = Dense(encoding_dim, activation='relu', kernel_initializer='he_normal')(input_layer)
encoder = Model(inputs=input_layer, outputs=encoder_layer)

# 自编码器模型
autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(Dense(input_dim, activation='sigmoid'))  # 解码层

autoencoder.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error')



# 训练自编码器模型，使用训练集的特征进行训练和验证
autoencoder.fit(features_train, features_train, epochs=100,batch_size=512, shuffle=True, validation_data=(features_test, features_test))

# 使用编码器对特征进行降维
encoded_features_train = encoder.predict(features_train)
encoded_features_test = encoder.predict(features_test)

# 使用编码器对特征进行降维
encoded_features_train = encoder.predict(features_train)

# 转换为Pandas DataFrame
encoded_features_train_df = pd.DataFrame(encoded_features_train)


# 使用降维后的特征训练预测模型，这里我们使用一个简单的线性回归模型作为例子
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(encoded_features_train, target_train)

# 对测试集进行预测并评估模型性能
predictions = regressor.predict(encoded_features_test)

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(target_test, predictions)
print('Mean Squared Error on test set: ', mse)

from sklearn.linear_model import LinearRegression

# 使用降维后的特征训练线性回归模型
regressor = LinearRegression()
regressor.fit(encoded_features_train, target_train)

# 获取线性回归模型的参数
coefficients = regressor.coef_  # 回归系数，对应每个特征的权重
intercept = regressor.intercept_  # 截距项

# 打印每个特征的回归系数
for i, coef in enumerate(coefficients):
    print(f'Coefficient for encoded feature {i+1}: {coef}')

# 打印截距项
print(f'Intercept: {intercept}')

import statsmodels.api as sm

# 添加常数项，对应线性回归模型的截距
encoded_features_train_with_intercept = sm.add_constant(encoded_features_train)

# 使用statsmodels的OLS函数训练线性回归模型
model = sm.OLS(target_train, encoded_features_train_with_intercept)
results = model.fit()

# 打印每个特征的回归系数、标准误差和p值
print(results.summary())


Epoch 1/100
123/123 [==============================] - 0s 3ms/step - loss: 0.1646 - val_loss: 0.1637
Epoch 2/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1626 - val_loss: 0.1610
Epoch 3/100
123/123 [==============================] - 0s 3ms/step - loss: 0.1590 - val_loss: 0.1565
Epoch 4/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1533 - val_loss: 0.1496
Epoch 5/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1453 - val_loss: 0.1407
Epoch 6/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1358 - val_loss: 0.1307
Epoch 7/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1254 - val_loss: 0.1200
Epoch 8/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1145 - val_loss: 0.1090
Epoch 9/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1034 - val_loss: 0.0977
Epoch 10/100
123/123 [==============================] - 0s 2ms/step - loss: 0.0921 - val_lo

In [8]:
import numpy as np
import pandas as pd
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras import regularizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures

# 数据整理
data = pd.read_csv('/Users/xiaoquanliu/Desktop/Book_DataCode/第七章/DL_Data1.csv')
data1=data.dropna()

features = data1.iloc[:, :-1] 
target = data1.iloc[:, -1]
from sklearn.preprocessing import PolynomialFeatures


# 创建一个 PolynomialFeatures 对象，设置 degree=2 来生成两两交互的特征
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

# 使用 PolynomialFeatures 对象转换你的数据
features_poly = poly.fit_transform(features)

# 将交互特征转换为 DataFrame，并添加列名
features_poly_df = pd.DataFrame(features_poly, columns=poly.get_feature_names_out(features.columns))

# 打印结果
print(features_poly_df)

# features_poly_df 包含了原始特征和它们的两两交互项

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features_poly_df)
features_train, features_test, target_train, target_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

input_dim = features_train.shape[1]  

         Return  Volatility     Beta1     Beta2      Cor1      Cor2  \
0      0.200627    0.583365  0.716008  0.726709  0.184198  0.188873   
1      0.200501    0.383995  0.913849  0.923003  0.409555  0.416662   
2      0.200431    0.423533  0.806881  0.831737  0.278495  0.292493   
3      0.200397    0.518040  0.943050  0.929599  0.254375  0.254799   
4      0.200378    0.550201  1.066480  1.068627  0.284123  0.290532   
...         ...         ...       ...       ...       ...       ...   
78552  0.000139    0.555577  0.944254  0.961975  0.247322  0.256579   
78553  0.000138    0.471594  1.882598  1.846909  0.568810  0.567869   
78554  0.000133    0.649889  1.180658  1.213428  0.317196  0.328823   
78555  0.000115    0.632866  1.286845  1.315512  0.349781  0.361143   
78556  0.000100    0.645599  1.472637  1.493594  0.335231  0.346022   

       NonSysRisk1  NonSysRisk2      Rsq1      Rsq2  ...  \
0         0.012349     0.012984  0.033929  0.035673  ...   
1         0.025676     0.02

In [9]:
#设置隐藏因子的数量
encoding_dim = 60
hidden_dim=315
num_hidden_layer=10


# 添加L1正则化
encoder_layer = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(0.01 ))

# 添加L2正则化
encoder_layer = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l2(0.01))

decoding_dim=256
input_layer = Input(shape=(630,))
encoder_layer = Dense(encoding_dim, activation='relu', kernel_initializer='he_normal')(input_layer)
encoder = Model(inputs=input_layer, outputs=encoder_layer)

# 自编码器模型
autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(Dense(input_dim, activation='sigmoid'))  # 解码层

autoencoder.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error')



# 训练自编码器模型，使用训练集的特征进行训练和验证
autoencoder.fit(features_train, features_train, epochs=100,batch_size=512, shuffle=True, validation_data=(features_test, features_test))

# 使用编码器对特征进行降维
encoded_features_train = encoder.predict(features_train)
encoded_features_test = encoder.predict(features_test)

# 使用编码器对特征进行降维
encoded_features_train = encoder.predict(features_train)

# 转换为Pandas DataFrame
encoded_features_train_df = pd.DataFrame(encoded_features_train)


# 使用降维后的特征训练预测模型，这里我们使用一个简单的线性回归模型作为例子
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(encoded_features_train, target_train)

# 对测试集进行预测并评估模型性能
predictions = regressor.predict(encoded_features_test)

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(target_test, predictions)
print('Mean Squared Error on test set: ', mse)

from sklearn.linear_model import LinearRegression

# 使用降维后的特征训练线性回归模型
regressor = LinearRegression()
regressor.fit(encoded_features_train, target_train)

# 获取线性回归模型的参数
coefficients = regressor.coef_  # 回归系数，对应每个特征的权重
intercept = regressor.intercept_  # 截距项

# 打印每个特征的回归系数
for i, coef in enumerate(coefficients):
    print(f'Coefficient for encoded feature {i+1}: {coef}')

# 打印截距项
print(f'Intercept: {intercept}')

import statsmodels.api as sm

# 添加常数项，对应线性回归模型的截距
encoded_features_train_with_intercept = sm.add_constant(encoded_features_train)

# 使用statsmodels的OLS函数训练线性回归模型
model = sm.OLS(target_train, encoded_features_train_with_intercept)
results = model.fit()

# 打印每个特征的回归系数、标准误差和p值
print(results.summary())


Epoch 1/100
123/123 [==============================] - 1s 3ms/step - loss: 0.1709 - val_loss: 0.1691
Epoch 2/100
123/123 [==============================] - 0s 3ms/step - loss: 0.1669 - val_loss: 0.1641
Epoch 3/100
123/123 [==============================] - 0s 3ms/step - loss: 0.1605 - val_loss: 0.1561
Epoch 4/100
123/123 [==============================] - 0s 3ms/step - loss: 0.1509 - val_loss: 0.1451
Epoch 5/100
123/123 [==============================] - 0s 3ms/step - loss: 0.1388 - val_loss: 0.1319
Epoch 6/100
123/123 [==============================] - 0s 3ms/step - loss: 0.1248 - val_loss: 0.1173
Epoch 7/100
123/123 [==============================] - 0s 3ms/step - loss: 0.1099 - val_loss: 0.1023
Epoch 8/100
123/123 [==============================] - 0s 3ms/step - loss: 0.0950 - val_loss: 0.0877
Epoch 9/100
123/123 [==============================] - 0s 3ms/step - loss: 0.0809 - val_loss: 0.0742
Epoch 10/100
123/123 [==============================] - 0s 3ms/step - loss: 0.0682 - val_lo

In [5]:
#设置隐藏因子的数量
encoding_dim = 30
hidden_dim=315
num_hidden_layer=20


# 添加L1正则化
encoder_layer = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(0.01 ))

# 添加L2正则化
encoder_layer = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l2(0.01))

decoding_dim=256
input_layer = Input(shape=(630,))
encoder_layer = Dense(encoding_dim, activation='relu', kernel_initializer='he_normal')(input_layer)
encoder = Model(inputs=input_layer, outputs=encoder_layer)

# 自编码器模型
autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(Dense(input_dim, activation='sigmoid'))  # 解码层

autoencoder.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error')



# 训练自编码器模型，使用训练集的特征进行训练和验证
autoencoder.fit(features_train, features_train, epochs=100,batch_size=512, shuffle=True, validation_data=(features_test, features_test))

# 使用编码器对特征进行降维
encoded_features_train = encoder.predict(features_train)
encoded_features_test = encoder.predict(features_test)

# 使用编码器对特征进行降维
encoded_features_train = encoder.predict(features_train)

# 转换为Pandas DataFrame
encoded_features_train_df = pd.DataFrame(encoded_features_train)


# 使用降维后的特征训练预测模型，这里我们使用一个简单的线性回归模型作为例子
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(encoded_features_train, target_train)

# 对测试集进行预测并评估模型性能
predictions = regressor.predict(encoded_features_test)

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(target_test, predictions)
print('Mean Squared Error on test set: ', mse)

from sklearn.linear_model import LinearRegression

# 使用降维后的特征训练线性回归模型
regressor = LinearRegression()
regressor.fit(encoded_features_train, target_train)

# 获取线性回归模型的参数
coefficients = regressor.coef_  # 回归系数，对应每个特征的权重
intercept = regressor.intercept_  # 截距项

# 打印每个特征的回归系数
for i, coef in enumerate(coefficients):
    print(f'Coefficient for encoded feature {i+1}: {coef}')

# 打印截距项
print(f'Intercept: {intercept}')

import statsmodels.api as sm

# 添加常数项，对应线性回归模型的截距
encoded_features_train_with_intercept = sm.add_constant(encoded_features_train)

# 使用statsmodels的OLS函数训练线性回归模型
model = sm.OLS(target_train, encoded_features_train_with_intercept)
results = model.fit()

# 打印每个特征的回归系数、标准误差和p值
print(results.summary())


Epoch 1/100
123/123 [==============================] - 0s 3ms/step - loss: 0.1633 - val_loss: 0.1619
Epoch 2/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1600 - val_loss: 0.1577
Epoch 3/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1550 - val_loss: 0.1520
Epoch 4/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1485 - val_loss: 0.1446
Epoch 5/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1404 - val_loss: 0.1358
Epoch 6/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1310 - val_loss: 0.1259
Epoch 7/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1206 - val_loss: 0.1152
Epoch 8/100
123/123 [==============================] - 0s 2ms/step - loss: 0.1095 - val_loss: 0.1038
Epoch 9/100
123/123 [==============================] - 0s 2ms/step - loss: 0.0982 - val_loss: 0.0926
Epoch 10/100
123/123 [==============================] - 0s 2ms/step - loss: 0.0874 - val_lo